In [1]:
import xml.sax
import re
from collections import defaultdict
from nltk.corpus import stopwords
import Stemmer
import time

In [2]:
count_of_pages = 0
Identities = {}
start_time = time.time()
stop_words=set(stopwords.words('english'))
ss = Stemmer.Stemmer('english') 

In [3]:
def get_infobar(data):
    data=data.split('\n')
    length=len(data)
    ret=[]
    flag=0
    for i in range(length):
        if flag%2==0 and re.search(r'\{\{infobox',data[i]):
            flag=1
            ret.append(re.sub(r'\{\{infobox','',data[i]))
        elif flag%2==1:
            if data[i]=='}}':
                flag=flag+1
                continue
            ret.append(data[i])
        elif flag==0:
            if i*100/length > 50:
                break

    return final_processing(' '.join(ret))

def get_categories(data):
    data=re.findall(r'\[\[category:.*\]\]',data)
    ret=[]
    for i in data:
        ret.append(i[11:len(i)-2])

    return final_processing(' '.join(ret))

def get_external_links(text):
    data=text.split('==External links==')
    if len(data)==1:
        return []
    
    data=data[1]
    data=re.findall(r'\*\s*\[.*\]',data)
    ret=[]
    for i in data:
        ret.append(i[2:len(i)-1])
    return final_processing(' '.join(ret))

def get_references(text):
    ret=[]
    data1=re.findall(r'\|\s*title[^\|]*',text)
    for i in data1:
        ret.append(i[i.find('=')+1:len(i)-1])
    return final_processing(' '.join(ret))

In [4]:
def final_processing(data):
    data=data.strip().encode("ascii",errors="ignore").decode()
    data=re.sub(r'\`|\~|\!|\@|\#|\"|\'|\$|\%|\^|\&|\*|\(|\)|\-|\_|\=|\+|\\|\||\]|\[|\}|\{|\;|\:|\/|\?|\.|\>|\,|\<|\'|\n|\||\|\/"',r' ',data)
    data=re.sub(r'&nbsp;|&lt;|&gt;|&amp;|&quot;|&apos;|&cent;|&pound;|&yen;|&euro;|&copy;|&reg;',r' ',data)
    
    final_data=[]
    
    data=data.split()
    for w in data:
        if not w.strip() in stop_words:
            w=ss.stemWord(w.lower())
            final_data.append(w)
    return final_data

In [5]:
def process_text(text,title):
    text=text.lower()
    references=[]
    categories=[]
    external_links=[]
    data=text.split('==References==')
    if data[0]==text:
        categories=get_categories(data[0])
        external_links=get_external_links(data[0])
    else:
        categories=get_categories(data[1])
        external_links=get_external_links(data[1])
        references=get_references(data[1])

    info_bar=get_infobar(data[0])
    data[0]=re.sub(r'\{\{.*\}\}',r' ',data[0])
    body=final_processing(data[0])
    title=final_processing(title.lower())
    return title,body,info_bar,categories,external_links,references

In [6]:
class MyParser(xml.sax.ContentHandler):
    def __init__(self):
        self.title = ""
        self.tag = ""
        self.id = ""
        self.text = ""
        self.idFlag = 0 
    def characters(self, content):
        if self.tag == 'text':
            self.text+=content
        if self.tag == 'title':
            self.title+=content
        if self.tag == 'id' and self.idFlag==0:
            self.id = content
            self.idFlag=1
    def startElement(self,name,attrs):
        self.tag=name

    def endElement(self,name):
        global count_of_pages
        if name=='page':
            self.title=self.title.strip().encode("ascii",errors="ignore").decode()
            Identities[count_of_pages]=self.title
            title,body,infobox,categories,external_links,references=process_text(self.text,self.title)
            count_of_pages += 1
            self.tag=""
            self.title=""
            self.text=""
            self.id=""
            self.idFlag=0

In [7]:
def main():
    global count_of_pages
    global Identities
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces, False)
    handler = MyParser()
    parser.setContentHandler(handler)
    output = parser.parse('../example.xml')
    print(count_of_pages)

In [8]:
main()

2599
